In [23]:
from typing import Optional, List


class State:
    itemNames: tuple[str, ...] = tuple("Itemnames not set")

    def __init__(self, itemsLeft: List[bool], prev: Optional["State"] = None):
        self.itemsLeft = itemsLeft
        self.prev = prev

    @classmethod
    def addItemNames(cls, itemNames: tuple[str, ...]):
        cls.itemNames = itemNames

    def __str__(self) -> str:
        return (
            "["
            + ", ".join(
                [
                    self.itemNames[i]
                    for i in range(len(self.itemNames))
                    if self.itemsLeft[i]
                ]
            )
            + "]"
        )

    def __eq__(self, other) -> bool:
        if isinstance(other, State):
            return self.itemsLeft == other.itemsLeft
        return False

    def __hash__(self):
        # This allows the state to be used in a set or as a dictionary key
        return hash(tuple(self.itemsLeft))

    def getNeighbours(self) -> List["State"]:

        neighbours: List[State] = []
        prev: State = self

        # farmer is on the right
        if not self.itemsLeft[0]:
            self.itemsLeft[0] = True
            neighbour = State(self.itemsLeft.copy(), prev)
            neighbours.append(neighbour)

            for i in range(1, len(self.itemsLeft)):
                if not self.itemsLeft[i]:
                    self.itemsLeft[i] = True
                    neighbour = State(self.itemsLeft.copy(), prev)
                    neighbours.append(neighbour)
                    self.itemsLeft[i] = False
            self.itemsLeft[0] = False

        # farmer is on the left
        if self.itemsLeft[0]:
            self.itemsLeft[0] = False
            neighbour = State(self.itemsLeft.copy(), prev)
            neighbours.append(neighbour)

            for i in range(1, len(self.itemsLeft)):
                if self.itemsLeft[i]:
                    self.itemsLeft[i] = False
                    neighbour = State(self.itemsLeft.copy(), prev)
                    neighbours.append(neighbour)
                    self.itemsLeft[i] = True
            self.itemsLeft[0] = True

        return neighbours


In [24]:
from typing import List, Set, Optional, Deque
from collections import deque
from collections.abc import Iterable


class FarmerGame:

    def __init__(
        self, itemNames: tuple[str, ...], badStates: Optional[Set[State]] = None
    ) -> None:
        self.itemNames = itemNames
        if not badStates:
            self.badStates: Set[State] = set()
        else:
            self.badStates = badStates
        self.source: Optional[State] = None
        self.target: Optional[State] = None

    def setSource(self, source: State) -> None:
        if source in self.badStates:
            raise ValueError("Source State is a bad State")
        else:
            self.source = source

    def setTarget(self, target: State) -> None:
        if target in self.badStates:
            raise ValueError("target State is a bad State")
        else:
            self.target = target

    def addBadStates(self, badStates: Iterable[State]) -> None:
        for State in badStates:
            self.badStates.add(State)

    def __backTrack(self, state: State) -> tuple[List[State], bool]:
        path: List[State] = [state]
        while state.prev:
            path.append(state.prev)
            state = state.prev
        return path[::-1], True

    def __movesFromPath(self, path: List[State]) -> tuple[List[State], bool]:
        print(f"Actions required to reach State {self.target} from {self.source}:")

        for i in range(len(path) - 1):
            curr: State = path[i]
            next: State = path[i + 1]
            leftIdx: List[int] = []
            rightIdx: List[int] = []

            for idx, values in enumerate(zip(curr.itemsLeft, next.itemsLeft)):
                diff: int = int(values[1]) - int(values[0])
                if diff == 0:
                    continue
                elif diff == 1:
                    leftIdx.append(idx)
                elif diff == -1:
                    rightIdx.append(idx)

            leftMovedItems: str = ", ".join([curr.itemNames[idx] for idx in leftIdx])
            rightMovedItems: str = ", ".join([curr.itemNames[idx] for idx in rightIdx])
            if leftMovedItems:
                print(f"Move {leftMovedItems} left")
            elif rightMovedItems:
                print(f"move {rightMovedItems} right")

        return path, True

    def bfs(self, printActions: bool = False) -> tuple[List[State], bool]:

        if not isinstance(self.source, State):
            raise ValueError("Source is not specified")
        if not isinstance(self.target, State):
            raise ValueError("Target is not specified")
        q: Deque[State] = deque()
        visited: Set[State] = set([self.source])
        q.append(self.source)

        while q:
            curr: State = q.popleft()
            if curr.itemsLeft == self.target.itemsLeft:
                path: List[State]
                succes: bool
                path, succes = self.__backTrack(curr)
                if printActions:
                    return self.__movesFromPath(path)
                else:
                    return path, succes

            for neighbour in curr.getNeighbours():
                if neighbour not in visited:
                    if neighbour not in self.badStates:
                        visited.add(neighbour)
                        q.append(neighbour)

        return [], False


In [25]:
from typing import List, Set


def gameReader(path: str) -> FarmerGame:

    boolMapping = lambda x: x == "1"

    with open(path, "r") as file:
        lines = file.readlines()
        itemNames: tuple[str, ...] = tuple(lines[0].split())
        game = FarmerGame(itemNames)
        State.addItemNames(itemNames)
        source: State = State(list(map(boolMapping, lines[1].split())))
        target: State = State(list(map(boolMapping, lines[2].split())))
        game.setSource(source)
        game.setTarget(target)
    return game


def badStateReader(path: str) -> Set[State]:
    boolMapping = lambda x: x == "1"
    with open(path, "r") as file:
        lines = file.readlines()
        nBadStates: int = int(lines[0])
        badStates: set[State] = set()
        for i in range(nBadStates):
            state = State(list(map(boolMapping, lines[i + 1].split())))
            badStates.add(state)
    return badStates


In [26]:
from typing import Set


if __name__ == "__main__":

    game: FarmerGame = gameReader("./data/data.txt")
    badStates: Set[State] = badStateReader("./data/badStates.txt")
    game.addBadStates(badStates)
    path, result = game.bfs(printActions=True)
    print(f"starting state of the game: {game.source}")
    print(f"ending state of the game: {game.target}")
    if result:
        print(
            f"path from start to end found with {len(path) - 1} steps: {list(map(str, path))}"
        )
    else:
        print(f"No path from {game.source} to {game.target} found")


Actions required to reach State [Farmer, Wolf, Goat, Cabbage] from []:
Move Farmer, Goat left
move Farmer right
Move Farmer, Wolf left
move Farmer, Goat right
Move Farmer, Cabbage left
move Farmer right
Move Farmer, Goat left
starting state of the game: []
ending state of the game: [Farmer, Wolf, Goat, Cabbage]
path from start to end found with 7 steps: ['[]', '[Farmer, Goat]', '[Goat]', '[Farmer, Wolf, Goat]', '[Wolf]', '[Farmer, Wolf, Cabbage]', '[Wolf, Cabbage]', '[Farmer, Wolf, Goat, Cabbage]']
